# #. Group Units

A Group is a collection of Group Units. A Group Unit consists of:
- **Nonce**: A unique identifier for the Group Unit.  Also, declares the Group Unit's Hierarchical Deterministic (HD) path.
- **Owner**: The owner of the Group Unit.  Generally, the owner is the Group's creator, but alternative ownership models are possible.
- **Credential**: A credential that is used to authenticate the Group Unit and allow access to the Group Unit's data.
- **Data**: The data that is stored in the Group Unit.

In [1]:
import sys

sys.path.append("../../forme-groups-python-3-12/")

## Controller

The Controller is the entity that creates the Group.  The Controller is responsible for:
- Creating the Group.
- Creating the Group Units.

In [2]:
from src.groups.controller import Controller
from src.groups.unit import Data

# Create a Controller with a fresh group unit pool
controller = Controller()

# Create some data to store in the Group
data = Data((1, 2, 3))

# Create a GroupUnit and store the data in it
group_unit = controller._create_group_unit(data)

print(group_unit)

nonce: 1, owner: , credential: , data: entry: (1, 2, 3), schema: None


## Create Sub Units from a schema



In [3]:
import uuid
import random
from src.groups.base import SchemaEntry, BaseSchema

# Create a SchemaEntry for the GroupUnit
schema_entry = SchemaEntry("name", "str")
schema_entry2 = SchemaEntry("age", "int")
schema_for_sub_units = BaseSchema((schema_entry, schema_entry2))

# Create the data to store in the GroupUnit
data = Data(('a', 'b', 'c'), schema_for_sub_units)

# Create a GroupUnit and store the data in it
group_unit = controller._create_group_unit(data)
print(group_unit)

# Create the data with the proper schema
sub_unit_data = Data(("Fred", 12))

sub_group_unit = controller._create_group_unit(sub_unit_data, True)

print(sub_group_unit)

sub_unit_data2 = Data(("Jane", 11))

sub_group_unit2 = controller._create_group_unit(sub_unit_data2)

print(sub_group_unit2)

sub_unit3 = Data(("Jill", 10))
# print(sub_unit3)
sub_group_unit3 = controller._create_group_unit(sub_unit3)
print(sub_group_unit3)

for i in range(0, 100):
    random_data = Data((str(uuid.uuid4()), random.randint(0, 100)))
    # print(controller.pool.group_units)
    # print(random_data)
    random_group_unit = controller._create_group_unit(random_data)
    # print(random_group_unit)

print(controller.pool.group_units[-1][2])


nonce: 2, owner: , credential: , data: entry: ('a', 'b', 'c'), schema: (key=name, value=str), (key=age, value=int)
nonce: 2.0, owner: , credential: , data: entry: ('Fred', 12), schema: None
nonce: 2.1, owner: , credential: , data: entry: ('Jane', 11), schema: None
nonce: 2.2, owner: , credential: , data: entry: ('Jill', 10), schema: None
nonce: 2.102, owner: , credential: , data: entry: ('f42b7e80-716f-42b8-9e26-5b12afc90c5b', 36), schema: None


In [4]:
new_controller = Controller()

new_base_schema = BaseSchema((SchemaEntry("name", "str"), SchemaEntry("age", "int"), SchemaEntry("is_valid", "bool")))

new_data_schema = Data(("Users"), new_base_schema)

new_group_unit = new_controller._create_group_unit(new_data_schema)

print(new_group_unit)

new_sub_unit_data = Data(("Fred", 12, True))

new_sub_group_unit = new_controller._create_group_unit(new_sub_unit_data, True, new_group_unit.data.schema)

print(new_sub_group_unit)

new_sub_unit_data2 = Data(("Jane", 11, False))

try:
    new_controller._create_group_unit(new_sub_unit_data2, True, new_group_unit.data.schema)
except Exception as e:
    print(e)

new_sub_unit_data3 = Data(("Jill", 10, False))

new_sub_group_unit3 = new_controller._create_group_unit(new_sub_unit_data3, False)

print(new_sub_group_unit3)

another_sub_schema = BaseSchema((SchemaEntry("title", "str"),))

another_sub_data = Data(("Aaron", 30, True), another_sub_schema)

another_sub_group_unit = new_controller._create_group_unit(another_sub_data, False, new_group_unit.data.schema)

print(another_sub_group_unit)

sub_sub_data = Data(("CEO",))

sub_sub_group_unit = new_controller._create_group_unit(sub_sub_data, True, another_sub_group_unit.data.schema)
print(sub_sub_group_unit)

nonce: 1, owner: , credential: , data: entry: ('Users',), schema: (key=name, value=str), (key=age, value=int), (key=is_valid, value=bool)
nonce: 1.0, owner: , credential: , data: entry: ('Fred', 12, True), schema: None
Cannot create a sub Unit without a schema
nonce: 1.1, owner: , credential: , data: entry: ('Jill', 10, False), schema: None
nonce: 1.2, owner: , credential: , data: entry: ('Aaron', 30, True), schema: (key=title, value=str)
